In [1]:
import torch
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import glob
from tqdm import tqdm

In [3]:
predictionDB = pd.read_csv("../data/processed/predictionDB.csv",lineterminator='\n')
embeddings = np.load('./../data/processed/embeddings/0003349d47ee039e4600f75f0b7c893aa63e481e.npy')


embeddings = [None]*len(predictionDB)
i=0
for np_name in glob.glob('./../data/processed/embeddings/*.np[yz]'):
    embeddings[i] = np.load(np_name)
    i = i + 1

embeddings
print(len(predictionDB),i)

62917 62917


In [4]:
embeddings2 = pd.Series( (v for v in embeddings) )
embeddings2

0        [-0.0032682128, -0.07904689, -0.03000979, 0.00...
1        [0.009278303, 0.059015848, -0.028337339, -0.00...
2        [-0.051631384, 0.014555246, -0.052139774, 0.04...
3        [0.03008972, 0.024424704, 0.024788633, -0.0183...
4        [-0.03437451, 0.054410815, -0.058744326, 0.014...
                               ...                        
62912    [-0.021051873, 0.00758308, 0.01212802, 0.03859...
62913    [-0.06524257, 0.0017218918, -0.079527594, -0.0...
62914    [-0.006718385, 0.048148204, -0.07988764, 0.043...
62915    [-0.057739496, 0.0742897, 0.019133838, -0.0250...
62916    [-0.054419886, -0.0072614155, -0.07316753, -0....
Length: 62917, dtype: object

In [5]:
from sklearn.model_selection import train_test_split

#data = embeddings
labels = predictionDB["inc_complexity"]

data_train, data_test, labels_train, labels_test = train_test_split(embeddings2, labels, test_size=0.20, random_state=42)


In [6]:
type(embeddings2)

pandas.core.series.Series

In [7]:
class commits_dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X.index)
    
    def __getitem__(self, index):
        return torch.Tensor(self.X.iloc[index]),torch.as_tensor(self.y.iloc[index]).float()

In [8]:
commits_dataset_train = commits_dataset(X=data_train,y=labels_train)
commits_dataset_test = commits_dataset(X=data_test,y=labels_test)
print(commits_dataset_train[0])

(tensor([-3.5666e-02,  2.1958e-02, -3.9789e-02, -6.1809e-03,  6.8587e-02,
        -7.5999e-02, -5.8423e-03, -2.8350e-02,  4.5228e-02,  4.2811e-02,
         4.8445e-03,  5.7028e-02, -1.8885e-02,  7.1698e-03, -5.3966e-02,
        -3.9067e-02, -6.4273e-02,  8.5946e-03, -4.0308e-02,  8.6781e-02,
        -1.5623e-01, -4.9236e-02,  1.1501e-01,  1.4198e-02,  2.9187e-02,
        -1.8272e-02, -3.0727e-02,  7.7440e-03, -4.8560e-02, -8.0119e-02,
         6.4895e-02, -4.4259e-03,  1.1658e-02,  1.3960e-02,  1.9249e-02,
         7.9462e-02,  5.7870e-02, -8.3256e-02, -3.9049e-02,  8.4726e-02,
        -4.4668e-02,  2.2374e-02,  1.2167e-02, -3.5477e-02, -4.9882e-02,
         1.9489e-02,  4.9052e-02, -5.6177e-02,  4.0880e-02,  3.0927e-02,
        -6.3145e-02, -4.6525e-02,  6.5082e-02, -5.3560e-02, -5.4483e-03,
        -4.4613e-02, -2.2856e-02,  4.4417e-02,  2.7936e-02,  6.1374e-02,
         2.0405e-02, -7.4768e-03, -5.2619e-02, -4.5750e-02, -9.3046e-03,
         4.2945e-02, -4.9864e-02, -1.3173e-02, -3.

In [9]:
train_loader = DataLoader(dataset=commits_dataset_train, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=commits_dataset_test, batch_size=32, shuffle=False)
#dls = DataLoaders(train_loader,valid_loader)


In [10]:
#predictionDB["is_valid"] = np.zeros(len(predictionDB))
#for i in range(len(predictionDB["is_valid"])):
#    predictionDB["is_valid"][i] = 1 if random.random()<0.2 else 0 

In [11]:
#from fastai.text.all import *
#dls = TextDataLoaders.from_df(predictionDB, text_col='COMMIT_MESSAGE', label_col='inc_complexity', valid_col='is_valid')
#dls.show_batch(max_n=3)

In [12]:
# Multilayer perceptron
class MultilayerPerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(384, 1024, bias=True) 
        self.lin2 = nn.Linear(1024, 120, bias=True)
        self.lin3 = nn.Linear(120, 1, bias=True)

    def forward(self, xb):
        x = xb.float()
        #x = xb.view(250, -1)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return self.lin3(x)

In [13]:
#mlp_learner = Learner(data=data, model=MultilayerPerceptron(), loss_func=nn.CrossEntropyLoss(),metrics=accuracy)
#mlp_learner.fine_tune(20)

In [14]:
model = MultilayerPerceptron()
print(model)

MultilayerPerceptron(
  (lin1): Linear(in_features=384, out_features=1024, bias=True)
  (lin2): Linear(in_features=1024, out_features=120, bias=True)
  (lin3): Linear(in_features=120, out_features=1, bias=True)
)


In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss()

In [16]:
mean_train_losses = []
mean_valid_losses = []
valid_acc_list = []
epochs = 15

for epoch in range(epochs):
    model.train()
    
    train_losses = []
    valid_losses = []
    for i, (embeddings, labels) in tqdm(enumerate(train_loader)):
        
        optimizer.zero_grad()
        
        outputs = model(embeddings)
        loss = loss_fn(outputs.squeeze(0),labels)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
            
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (embeddings, labels) in enumerate(valid_loader):
            outputs = model(embeddings)
            loss = loss_fn(outputs.squeeze(0), labels)
            
            valid_losses.append(loss.item())
            
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            
    mean_train_losses.append(np.mean(train_losses))
    mean_valid_losses.append(np.mean(valid_losses))
    print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}'\
         .format(epoch+1, mean_train_losses[-1], mean_valid_losses[-1]))

0it [00:00, ?it/s]/home/elias/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
1569it [00:09, 159.89it/s]/home/elias/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
1573it [00:09, 161.44it/s]
/home/elias/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect resu

epoch : 1, train loss : 265371.8512, valid loss : 229812.7388


1573it [00:10, 146.17it/s]
15it [00:00, 145.07it/s]

epoch : 2, train loss : 265343.3577, valid loss : 229808.9588


1573it [00:10, 147.45it/s]
14it [00:00, 139.91it/s]

...

epoch : 13, train loss : 265341.6575, valid loss : 229800.1672


1573it [00:10, 147.12it/s]
15it [00:00, 146.90it/s]

epoch : 14, train loss : 265338.2993, valid loss : 229800.1691


1573it [00:10, 148.84it/s]


epoch : 15, train loss : 265344.9254, valid loss : 229800.0235
